In [1]:
import os
from pathlib import Path
import fnmatch
import json
import pandas as pd
import re

In [13]:
# for path,dirs,files in os.walk('.\\runs'):
#     for file in fnmatch.filter(files,'all_results.json'):
#         fullname = os.path.abspath(os.path.join(path,file))
        
#         task_name = os.path.basename(path)
#         folder_str = os.path.basename(os.path.dirname(os.path.dirname(fullname))) # move up two folders and get folder name
#         arguments = {}
#         if 'mod' in folder_str:
#             arguments['modded'] = True
#         else:
#             arguments['modded'] = False

#         rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", folder_str)
#         arguments["num_train_epochs"] = int(rr[1])
#         arguments["hidden_dropout"] = float(rr[0])

#         # print(arguments)
#         with open(os.path.abspath(os.path.join(path,'used_arguments.json')), "w") as f: 
#             json.dump(arguments,f)
        

IndexError: list index out of range

In [3]:
series_list = []
for path,dirs,files in os.walk('.\\runs'):
    for file in fnmatch.filter(files,'used_arguments.json'):
        with open(os.path.abspath(os.path.join(path,file))) as f:
            d = json.load(f)
            series_list.append(d)
            break

df = pd.DataFrame(series_list)
df

,task_name,train_file,validation_file,max_length,pad_to_max_length,model_name_or_path,use_slow_tokenizer,per_device_train_batch_size,per_device_eval_batch_size,learning_rate,...,checkpointing_steps,resume_from_checkpoint,with_tracking,report_to,ignore_mismatched_sizes,hidden_dropout,train_data_frac,use_pretrain_dropout_in_first_pass,use_modded,eval_accuracy
0,wnli,None,None,128,False,bert-base-cased,False,32,8,0.00005,...,None,None,True,tensorboard,False,0.1,1.0,True,True,0.436620
1,wnli,None,None,128,False,bert-base-cased,False,32,8,0.00005,...,None,None,True,tensorboard,False,0.1,1.0,True,True,0.281690
2,wnli,None,None,128,False,bert-base-cased,False,32,8,0.00005,...,None,None,True,tensorboard,False,0.1,1.0,True,True,0.112676
3,wnli,None,None,128,False,bert-base-cased,False,32,8,0.00005,...,None,None,True,tensorboard,False,0.1,1.0,True,False,0.549296
4,wnli,None,None,128,False,bert-base-cased,False,32,8,0.00005,...,None,None,True,tensorboard,False,0.1,1.0,True,False,0.464789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,rte,None,None,128,False,bert-base-cased,False,32,8,0.00001,...,None,None,True,tensorboard,False,0.8,0.1,False,True,0.451264
428,rte,None,None,128,False,bert-base-cased,False,32,8,0.00001,...,None,None,True,tensorboard,False,0.8,0.1,False,True,0.451264
429,rte,None,None,128,False,bert-base-cased,False,32,8,0.00001,...,None,None,True,tensorboard,False,0.8,0.1,False,False,0.454874
430,rte,None,None,128,False,bert-base-cased,False,32,8,0.00001,...,None,None,True,tensorboard,False,0.8,0.1,False,False,0.451264


In [4]:
keep = [c for c in list(df) if len(df[c].unique()) > 1]
df_small = df[keep]
df_small

,task_name,learning_rate,num_train_epochs,max_train_steps,output_dir,hidden_dropout,train_data_frac,use_pretrain_dropout_in_first_pass,use_modded,eval_accuracy
0,wnli,0.00005,5,100,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.1,1.0,True,True,0.436620
1,wnli,0.00005,10,200,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.1,1.0,True,True,0.281690
2,wnli,0.00005,20,400,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.1,1.0,True,True,0.112676
3,wnli,0.00005,5,100,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.1,1.0,True,False,0.549296
4,wnli,0.00005,10,200,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.1,1.0,True,False,0.464789
...,...,...,...,...,...,...,...,...,...,...
427,rte,0.00001,10,80,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.8,0.1,False,True,0.451264
428,rte,0.00001,20,160,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.8,0.1,False,True,0.451264
429,rte,0.00001,10,80,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.8,0.1,False,False,0.454874
430,rte,0.00001,5,40,/content/drive/MyDrive/Masterarbeit/fantastic-...,0.8,0.1,False,False,0.451264


In [5]:
df_small.to_csv(".\\runs\\summary.csv")

In [27]:
import dtale

dtale.show(df_small)

2023-06-14 23:58:57,398 - INFO     - Executing shutdown due to inactivity...
2023-06-14 23:59:13,667 - INFO     - Executing shutdown...
2023-06-14 23:59:13,669 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\sit